In [1]:
import numpy as np
from skimage import io
from pathlib import Path
import re
import ants
from skimage.transform import resize
from tqdm import tqdm
from skimage.morphology import skeletonize_3d, binary_closing
from scipy.ndimage import distance_transform_edt, binary_dilation
import tifffile as tif
from scipy.ndimage import binary_fill_holes
import cc3d
from scipy.io import loadmat, savemat
import sknw
import networkx as nx
import pickle
import os
import matplotlib.pyplot as plt
import pandas as pd
import time
import scipy as sp
import vg
from pytransform3d.rotations import matrix_from_axis_angle
import multiprocessing
from scipy.ndimage import convolve as conv
from scipy.stats import multivariate_normal
from skimage import color, data, restoration
from RedLionfishDeconv import doRLDeconvolutionFromNpArrays
from matplotlib.patches import Circle
from skimage.feature import peak_local_max
from statistics import mode
import imageio
from PIL import Image
from PIL.TiffTags import TAGS
from tifffile import TiffFile
from sklearn.metrics import normalized_mutual_info_score, adjusted_mutual_info_score
plt.rcParams['figure.figsize'] = [15, 15]

In [2]:
def _ants_affine_to_distance(affine):

    dx, dy, dz = affine[9:]

    rot_x = np.arcsin(affine[6])
    cos_rot_x = np.cos(rot_x)
    rot_y = np.arctan2(affine[7] / cos_rot_x, affine[8] / cos_rot_x)
    rot_z = np.arctan2(affine[3] / cos_rot_x, affine[0] / cos_rot_x)

    deg = np.degrees

    return dx, dy, dz, deg(rot_x), deg(rot_y), deg(rot_z)

In [3]:
folder = Path('../TH1-CHR2_Small_Volumes/Female1')
files  = list(folder.glob('*.tif'))
files

[PosixPath('../TH1-CHR2_Small_Volumes/Female1/128_cap_1.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/276_cap_2_0001.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/276_cap_2.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/57_art_2.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/57_art_2_0001.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/XYZ1.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/276_cap_3_0001.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/276_cap_3.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/304_arteriole_2.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/304_arteriole_2_0001.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/57_art_3.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/128_cap_1_0001.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/57_art_3_0001.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/128_cap_2_0001.tif'),
 PosixPath('../TH1-CHR2_Small_Volumes/Female1/304_arteriole_1_0001.tif'),
 Posix

In [4]:
mouse_ids_path = Path('../TH1-CHR2_Small_Volumes/Female1')#each mouse has its own folder with raw data in it
mouse_ids = list(mouse_ids_path.glob('*.tif'))#grab folder names/mouse ids
mouse_ids = sorted([x.as_posix() for x in mouse_ids])
data_dicts = [
    {"image":image_name}
    for image_name in mouse_ids
]

#data_dicts = [data_dicts[_i]]
re.sub('matt_raw_warped_upsampled','matt_preds_registered',data_dicts[0]["image"])

'../TH1-CHR2_Small_Volumes/Female1/128_cap_1.tif'

# Define connected componnet removal

In [5]:
def remove_small_comps_3d(image, thresh = 500):
    """

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img,thresh=1000):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp,thresh=thresh)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res

def _rotmat(vector, points):
    """
    Rotates a 3xn array of 3D coordinates from the +z normal to an
    arbitrary new normal vector.
    """
    
    vector = vg.normalize(vector)
    axis = vg.perpendicular(vg.basis.z, vector)
    angle = vg.angle(vg.basis.z, vector, units='rad')
    
    a = np.hstack((axis, (angle,)))
    R = matrix_from_axis_angle(a)
    
    r = sp.spatial.transform.Rotation.from_matrix(R)
    rotmat = r.apply(points)
    
    return rotmat

def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.sum((nodes - node)**2, axis=1)
    if np.min(dist_2)>10:
        return node
    else:
        return nodes[np.argmin(dist_2)]

# register raw iamges

In [7]:
folder = Path('../TH1-CHR2_Small_Volumes/Female1')
files  = sorted(list(folder.glob('*.tif')))[0:-1]
files = [x.as_posix() for x in files]
files

['../TH1-CHR2_Small_Volumes/Female1/128_cap_1.tif',
 '../TH1-CHR2_Small_Volumes/Female1/128_cap_1_0001.tif',
 '../TH1-CHR2_Small_Volumes/Female1/128_cap_2.tif',
 '../TH1-CHR2_Small_Volumes/Female1/128_cap_2_0001.tif',
 '../TH1-CHR2_Small_Volumes/Female1/276_cap_1.tif',
 '../TH1-CHR2_Small_Volumes/Female1/276_cap_1_0001.tif',
 '../TH1-CHR2_Small_Volumes/Female1/276_cap_2.tif',
 '../TH1-CHR2_Small_Volumes/Female1/276_cap_2_0001.tif',
 '../TH1-CHR2_Small_Volumes/Female1/276_cap_3.tif',
 '../TH1-CHR2_Small_Volumes/Female1/276_cap_3_0001.tif',
 '../TH1-CHR2_Small_Volumes/Female1/304_arteriole_1.tif',
 '../TH1-CHR2_Small_Volumes/Female1/304_arteriole_1_0001.tif',
 '../TH1-CHR2_Small_Volumes/Female1/304_arteriole_2.tif',
 '../TH1-CHR2_Small_Volumes/Female1/304_arteriole_2_0001.tif',
 '../TH1-CHR2_Small_Volumes/Female1/57_art_1.tif',
 '../TH1-CHR2_Small_Volumes/Female1/57_art_1_0001.tif',
 '../TH1-CHR2_Small_Volumes/Female1/57_art_2.tif',
 '../TH1-CHR2_Small_Volumes/Female1/57_art_2_0001.tif',

In [17]:
for file in tqdm(files):
    img = io.imread(file)
    for i in range(img.shape[0]):
        io.imsave(re.sub('Female1','Female1_slices',re.sub('.tif','_'+str(i)+'.tif',file)), img[i], check_contrast=False)

100%|██████████| 20/20 [02:26<00:00,  7.31s/it]


In [25]:
folder = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
files  = sorted(list(folder.glob('*.tif')))[0:-1]
files = [x.as_posix() for x in files]
files = [x for x in files if 'warped' not in x]
print(len(files))
files[0:5]

3062


['../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0.tif',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_0.tif',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_1.tif',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_10.tif',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_100.tif']

In [26]:
keys = ['128_cap',
        '276_cap',
        '304_arteriole',
        '57_art']

In [27]:
dic = {}
for key in keys:
        slices = [x for x in files if key in x]
        dic[slices[0]] = slices[1:]
        #scans = [x for x in scans if 'res' in str(x)]
        #bottoms_1 = df[key][df[key][df[key].columns[3]] == 500]
        #bottoms_2 = df[key][df[key][df[key].columns[2]] == 500]
        #bottoms = pd.concat((bottoms_1,bottoms_2))
        #bottoms = np.array(bottoms[bottoms.columns[1]])
        #bottoms = [addition + '/' + x for x in bottoms]
        #bottoms = [x for x in bottoms if 'res' in x]
        #tops_1 = df[key][df[key][df[key].columns[3]] == 0]
        #tops_2 = df[key][df[key][df[key].columns[2]] == 0]
        #tops = pd.concat((tops_1,tops_2))
        #tops = np.array(tops[tops.columns[1]])
        #tops = [addition + '/' + x for x in tops]
        #tops = [x for x in tops if 'res' in x]
        #if len(tops) > 1:
        #    dic[tops[0]] = list(tops[1:])
        #elif len(tops) == 1:
        #    dic[tops[0]] = tops
        #if len(bottoms) > 1:
        #    dic[bottoms[0]] = list(bottoms[1:])
        #elif len(bottoms) == 1:
        #    dic[bottoms[0]] = bottoms

In [28]:
dic.keys()

dict_keys(['../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0.tif', '../TH1-CHR2_Small_Volumes/Female1_slices/276_cap_1_0.tif', '../TH1-CHR2_Small_Volumes/Female1_slices/304_arteriole_1_0.tif', '../TH1-CHR2_Small_Volumes/Female1_slices/57_art_1_0.tif'])

In [29]:
#dic

In [30]:
#mouse_ids_path = Path('/home/rozakmat/projects/rrg-bojana/data/THY1-TBI')#each mouse has its own folder with raw data in it
#mouse_ids = list(mouse_ids_path.glob('*?[0-9]/*res*?[0-9].tif'))#grab folder names/mouse ids
#images = sorted([x.as_posix() for x in mouse_ids if '_0001' in x.as_posix()])
##images = [x for x in images if 'vbm' in x]
#images = [x for x in images if  any(y in x for y in list(dic.keys()))]
##images = [x for x in images if any(y in x for y in ['14/','49/','56/','68/','65/','61/'])]
#unused_keys = [x for x in list(dic.keys()) if not  any(x in y for y in images)]
#print(len(images))
#print(images[1])
#new_file_name = re.sub('matt_raw_warped_upsampled','matt_preds_registered',data_dicts[0]["image"])
##images

In [31]:
#mov_files

In [32]:
#np.random.shuffle(images)

In [33]:
res = []
for i in tqdm(dic.keys()):
        fix_file = i
    #if not os.path.exists(re.sub('.tif','_warped.tif',fix_file)):
        mov_files = dic[i]
        mov_files = [x for x in mov_files if os.path.exists(x)]
        mov_files = [x for x in mov_files if not os.path.exists(re.sub('.tif','_warped.tif',x))]
        #mov_files = sorted(mov_files + [re.sub('.tif','_0001.tif',x) for x in mov_files])
        #mov_files.append(re.sub('.tif','_0001.tif',fix_file))
        mov_files = [x for x in mov_files if x != fix_file]
        mov_files = sorted(mov_files)
        mov_files = np.unique(mov_files)
        print(fix_file)
        fix_numpy = io.imread(fix_file)
        #plt.imshow(np.max(fix_numpy[:,0],axis=0))
        #plt.show()
        fix = ants.from_numpy(np.float32(fix_numpy[:,0])) #convert images to ants 
        res2 = []
        for mov_file in tqdm(mov_files):
            # read baseline image
            mov_numpy = io.imread(mov_file) # read followup image
            mov = ants.from_numpy(np.float32(mov_numpy[:,0]))
            mytx = ants.registration(fixed = fix,
                                     moving = mov,
                                     type_of_transform = 'Rigid',
                                     total_sigma = 2,
                                     aff_metric = 'meansquares'
                                     ) # register images and get displacment
            warpedraw_1 = ants.apply_transforms(fixed = fix,
                                                moving = ants.from_numpy(np.float32(mov_numpy[:,0])),
                                                transformlist = mytx['fwdtransforms'],
                                                interpolator = 'linear'
                                                ) # move vascular chanel
            warpedraw_2 = ants.apply_transforms(fixed = fix,
                                                moving = ants.from_numpy(np.float32(mov_numpy[:,1])),
                                                transformlist = mytx['fwdtransforms'],
                                                interpolator = 'linear'
                                                ) # move neuron chanel
            mov_numpy[:,0,:,:] = warpedraw_1[:,:,:]
            mov_numpy[:,1,:,:] = warpedraw_2[:,:,:]#combine moved chanels int one image
            #plt.imshow(np.max(warpedraw_1[:,:,:],axis=0))
            #plt.show()
            res2.append(_ants_affine_to_distance(ants.read_transform(mytx['fwdtransforms'][0]).parameters))
            io.imsave(re.sub('.tif','_warped.tif',mov_file),mov_numpy, check_contrast=False)# save warped followup image and baseline image
        res.append(res2)
        io.imsave(re.sub('.tif','_warped.tif',fix_file),fix_numpy, check_contrast=False)

  0%|          | 0/4 [00:00<?, ?it/s]

../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0.tif



0it [00:00, ?it/s]
 25%|██▌       | 1/4 [00:07<00:22,  7.46s/it]

../TH1-CHR2_Small_Volumes/Female1_slices/276_cap_1_0.tif



0it [00:00, ?it/s]
 50%|█████     | 2/4 [00:22<00:23, 11.94s/it]

../TH1-CHR2_Small_Volumes/Female1_slices/304_arteriole_1_0.tif



0it [00:00, ?it/s]
 75%|███████▌  | 3/4 [00:31<00:10, 10.36s/it]

../TH1-CHR2_Small_Volumes/Female1_slices/57_art_1_0.tif



100%|██████████| 4/4 [00:51<00:00, 12.77s/it]


In [14]:
folder = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
files  = sorted(list(folder.glob('*.tif')))[0:-1]
files = [x.as_posix() for x in files]
files = [x for x in files if 'warped' in x]
scans = np.unique(['_'.join(x.split('/')[-1].split('_')[0:3]) for x in files])
print(len(files))
files[0:5]

3061


['../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_0_warped.tif',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_100_warped.tif',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_101_warped.tif',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_102_warped.tif',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_103_warped.tif']

In [67]:
'/'.join(file.split('/')[:-1])+'/'+scan+'.tif'
#io.imsave('/'.join(file.split('/')[:-1])+'/'+scan+'.tif', img)
re.sub('_slices','','/'.join(file.split('/')[:-1])+'/'+scan+'.tif')

'../TH1-CHR2_Small_Volumes/Female1/57_art_3.tif'

In [69]:
def grab_time(x):
    return(int(x.split('_')[-2]))

for scan in tqdm(scans):
    _files = [x for x in files if scan in x]
    _files_pre  = [x for x in _files if '_0001' not in x]
    _files_post  = [x for x in _files if '_0001' in x]
    _files_pre = sorted(_files_pre, key = grab_time)
    _files_post = sorted(_files_post, key = grab_time)
    img = io.imread( _files_pre[0])
    img = np.expand_dims(img, axis=0)
    shape = np.array(img.shape)
    shape[2] = 1
    new = np.zeros(shape)
    img = np.append(img,new,axis=2)
    for file in _files_pre[1:]:
        _img = io.imread(file)
        _img = np.expand_dims(_img, axis=0)
        _img = np.append(_img,new,axis=2)
        img = np.append(img,_img,axis=0)
    img = np.append(img,np.concatenate((new,new,np.ones(shape)*1023),axis=2),axis=0)
    for file in _files_post:
        _img = io.imread(file)
        _img = np.expand_dims(_img, axis=0)
        _img = np.append(_img,new,axis=2)
        img = np.append(img,_img,axis=0)
    io.imsave(re.sub('_slices','','/'.join(file.split('/')[:-1])+'/'+scan+'.tif'), img)
    print(re.sub('_slices','','/'.join(file.split('/')[:-1])+'/'+scan+'.tif'))
    print(img.shape)

In [57]:
img.shape

(229, 20, 3, 128, 512)

In [50]:
shape = np.array(img.shape)
shape[2] = 1
new = np.zeros(shape)
img = np.append(img,new,axis=2)
img.shape

(1, 20, 3, 128, 512)

In [54]:
np.max(img)

1023.0

In [53]:
np.concatenate((new,new,np.ones(shape)*1023),axis=2)

(1, 20, 3, 128, 512)

# predict using trained model
run unetr prediction with registered raw images, orediction will be in same coordinate system \
run predict_matt_warped.py via predict_small_volumes.sh

# Binarize prediction output

In [144]:
directory = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
files  = directory.glob('*mean.npy')
#files  = directory.glob('*warped.tif')
files = sorted([x.as_posix() for x in files])
#files = [x for x in files if not os.path.exists(re.sub('led/','led_seg/',re.sub('mean','seg',x)))]
#files = [x for x in files if 'Feb52021_6' in x]
print(len(files))

3062


In [145]:
#for i in files:
#    os.remove(i)

In [146]:
#directory = Path('matt_raw_warped_single_upsampled')
#files  = directory.glob('*-*_mean.npy')
#files = sorted([x.as_posix() for x in files])
##files = [x for x in files if not os.path.exists(re.sub('led/','led_seg/',re.sub('mean','seg',x)))]
##files = [x for x in files if 'Feb52021_6' in x]
##files = [x for x in files if any(y in x for y in problem)]
#print(len(files))

In [147]:
min_prob = 0.5
max_var = 0.2
#np.random.shuffle(files)
for file in tqdm(files):
    if not os.path.exists(re.sub('mean','seg',file)):
            #print(file)
            mean = np.load(file)
            seg = np.zeros(mean.shape[1:]).astype('int8')
            seg[(mean[1,:,:,:] > min_prob)] = 1
            seg[(mean[2,:,:,:] > min_prob)] = 2
            seg = seg.astype('int8')
            seg = (seg==1)*1
            seg = fill_holes(seg)
            seg = remove_small_comps_3d(seg)
            #plt.imshow(np.max(seg,axis=2))
            #plt.show()
            #print(seg.shape)
            np.save(re.sub('mean','seg',file),seg)

100%|██████████| 3062/3062 [04:06<00:00, 12.41it/s]


In [148]:
os.path.exists(re.sub('mean','std',file))

False

In [149]:
directory = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
files  = directory.glob('*_seg.npy')
files = sorted([x.as_posix() for x in files])
print(len(files))
files[0]

3062


'../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_0_warped_seg.npy'

# Get distance transform of neuron segmentation

In [105]:
#directory = Path('matt_raw_warped_single_upsampled')
#files  = directory.glob('*-*_mean.npy')
#files = sorted([x.as_posix() for x in files])
#print(len(files))

In [106]:
#min_prob = 0.75
#max_var = 0.1
#for file in tqdm(files[::-1]):
#    if not os.path.exists(re.sub('led/','led_seg/',re.sub('mean','seg_nrn_dst',file))):
#        if os.path.exists(re.sub('mean','2x_std',file)):
#            mean = np.load(file)
#            std = np.load(re.sub('mean','2x_std',file))
#            seg = np.zeros(mean.shape[1:])
#            seg[(mean[1,:,:,:] > min_prob) * (std[1,:,:,:] < max_var)] = 1
#            seg[(mean[2,:,:,:] > min_prob) * (std[2,:,:,:] < max_var)] = 2
#            seg = seg.astype('int8')
#            seg = (seg==2)*1
#            np.save(re.sub('led/','led_seg/',re.sub('mean','seg_nrn',file)),seg)
#            np.save(re.sub('led/','led_seg/',re.sub('mean','seg_nrn_dst',file)),distance_transform_edt(1-seg))

In [107]:
#re.sub('mean','seg_nrn_dst',file)

# Problem Segmentations at graph

In [108]:
#problem = [
#    "XYZres038",
#    "XYZres041",
#    "XYZres042",
#    "XYZres048",
#    "XYZres052",
#    "XYZres026",
#    "XYZres024"
#]

In [109]:
#dic.keys()

# get predicted images and save matlab .mat of intersection

In [28]:
directory = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
files  = directory.glob('*_seg.npy')
files = sorted([x.as_posix() for x in files])
print(len(files))
files[0]

3062


'../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_0_warped_seg.npy'

In [29]:
keys = ['128_cap',
        '276_cap',
        '304_arteriole',
        '57_art']

In [30]:
dic = {}
for key in keys:
        slices = [x for x in files if key in x]
        dic[slices[0]] = slices[1:]

Currently set to union as intersection makes it choppy, retraining unetr on zoomed in data

In [31]:
re.sub('_warped_seg.npy','_seg_warped_single_sing.mat',re.sub('_0001','',image))

NameError: name 'image' is not defined

In [32]:
res = []
for image in tqdm(list(dic.keys())):
    if not os.path.exists(re.sub('_warped_seg.npy','_seg_warped_single_sing.mat',re.sub('_0001','',image))):
        print(image)
        img = binary_dilation(np.load(image))
        _img = np.copy(img)
        key = image
        mut_info = []
        mov_files = dic[image]
        for i in tqdm(mov_files):
            img_0001 = binary_dilation(np.load(i))
            _mut_info = normalized_mutual_info_score(_img.flatten(),img_0001.flatten()) # calculate mutual information between masks, judges registration
            mut_info.append(_mut_info)
            if _mut_info > 0.2:
                img += img_0001
                #img = img * img_0001
                #print(i)
        seg = img
        seg[seg!=0] = seg[seg!=0]/seg[seg!=0]
        seg = (seg==1)*1
        seg = seg.astype('int8')
        seg = binary_closing(binary_closing(binary_closing(fill_holes(seg))))
        seg = remove_small_comps_3d(seg,thresh=250)
        #print(mut_info)
        #print(mov_files)
        plt.imshow(np.max(seg,axis=2))
        plt.show()
        res += mut_info
        savemat(re.sub('_warped_seg.npy','_seg_warped_single_sing.mat',re.sub('_0001','',image)),{'FinalImage':fill_holes(binary_closing(seg))})
        np.save(re.sub('_warped_seg.npy','_seg_warped_single_sing.npy',re.sub('_0001','',image)),fill_holes(binary_closing(seg)))

100%|██████████| 4/4 [00:00<00:00, 24.28it/s]


In [33]:
directory = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
files  = directory.glob('*_seg_warped_single_sing.mat')
files = sorted([x.as_posix() for x in files])

In [34]:
files

['../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0_seg_warped_single_sing.mat',
 '../TH1-CHR2_Small_Volumes/Female1_slices/276_cap_1_0_seg_warped_single_sing.mat',
 '../TH1-CHR2_Small_Volumes/Female1_slices/304_arteriole_1_0_seg_warped_single_sing.mat',
 '../TH1-CHR2_Small_Volumes/Female1_slices/57_art_1_0_seg_warped_single_sing.mat']

# Generate Graphs

In [35]:
directory = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
files_seg_0001 = directory.glob('*_skel_warped_single_sing.mat')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])

files_seg_0001

['../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0_skel_warped_single_sing.mat',
 '../TH1-CHR2_Small_Volumes/Female1_slices/276_cap_1_0_skel_warped_single_sing.mat',
 '../TH1-CHR2_Small_Volumes/Female1_slices/304_arteriole_1_0_skel_warped_single_sing.mat',
 '../TH1-CHR2_Small_Volumes/Female1_slices/57_art_1_0_skel_warped_single_sing.mat']

In [36]:
directory = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
files_seg_0001 = directory.glob('*_warped_seg.npy')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])
files_seg_0001 = [x for x in files_seg_0001 if  any(y in x for y in list(dic.keys()))]
np.random.shuffle(files_seg_0001)

len(files_seg_0001)


4

In [37]:
list(dic.keys())

['../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_0_warped_seg.npy',
 '../TH1-CHR2_Small_Volumes/Female1_slices/276_cap_1_0001_0_warped_seg.npy',
 '../TH1-CHR2_Small_Volumes/Female1_slices/304_arteriole_1_0001_0_warped_seg.npy',
 '../TH1-CHR2_Small_Volumes/Female1_slices/57_art_1_0001_0_warped_seg.npy']

In [38]:
#file

In [29]:
np.random.shuffle(files_seg_0001)
for file_0001 in tqdm(files_seg_0001[:]):
    #if not os.path.exists(re.sub('_warped_seg.npy','_warped.pickle',file_0001)) or (time.time() - os.path.getmtime(re.sub('_warped_seg.npy','_warped_sing.pickle',file_0001)))/3600>48:
        #if os.path.exists(re.sub('_0001_warped_seg.npy','_skel_warped_single_sing.mat',file_0001)):
            file = file_0001
            skel_file = re.sub('_warped_seg.npy','_skel_warped_single_sing.mat',re.sub('_0001','',file))
            skel = loadmat(skel_file)['FilteredImage']
            if np.sum(skel) != 0:
                graph = sknw.build_sknw(skel, multi=False)
                print(len(graph.edges))
                while len(list(nx.selfloop_edges(graph)))>0:
                    if len(list(nx.selfloop_edges(graph))) !=0:
                        for edge in list(nx.selfloop_edges(graph)):
                            skel[graph[edge[0]][edge[1]]['pts'][1:-1,0],graph[edge[0]][edge[1]]['pts'][1:-1,1],graph[edge[0]][edge[1]]['pts'][1:-1,2]] = 0
                    for edge in graph.edges:
                        if (graph.degree(edge[0]) == 1 and graph.degree(edge[1]) > 2) or (graph.degree(edge[1]) == 1 and graph.degree(edge[0]) > 2):
                            if graph[edge[0]][edge[1]]['weight']<20:
                                skel[graph[edge[0]][edge[1]]['pts'][1:-1,0],graph[edge[0]][edge[1]]['pts'][1:-1,1],graph[edge[0]][edge[1]]['pts'][1:-1,2]] = 0
                    graph = sknw.build_sknw(skel, multi=False)
                for edge in graph.edges:
                    if (graph.degree(edge[0]) == 1 and graph.degree(edge[1]) > 2) or (graph.degree(edge[1]) == 1 and graph.degree(edge[0]) > 2):
                        if graph[edge[0]][edge[1]]['weight']<20:
                            skel[graph[edge[0]][edge[1]]['pts'][1:-1,0],graph[edge[0]][edge[1]]['pts'][1:-1,1],graph[edge[0]][edge[1]]['pts'][1:-1,2]] = 0
                graph = sknw.build_sknw(skel, multi=False)
                graph.remove_nodes_from(list(nx.isolates(graph)))
                print(len(graph.edges))
                io.imsave(re.sub('_warped_seg.npy','_single_skel_sing.tif',file),skel)
                nx.write_gpickle(graph,re.sub('_warped_seg.npy','_warped.pickle',file))
                key = [x for x in list(dic.keys()) if x in file][0]
                mov_files = [re.sub(key,x,file) for x in dic[key]]
                mov_files = sorted(mov_files + [re.sub('_0001','',x) for x in mov_files])
                mov_files.append(re.sub('_0001','',file))
                seg = np.load(re.sub('_0001','',file))
                for _file in mov_files:
                    if os.path.exists(_file):
                        if not os.path.exists(re.sub('_warped_seg.npy','_warped.pickle',_file)):
                            seg_0001 = np.load(_file)
                            _mut_info = normalized_mutual_info_score(seg.flatten(),seg_0001.flatten()) # calculate mutual information between masks, judges registration
                            if _mut_info > 0.1:
                                print('matched')
                                nx.write_gpickle(graph,re.sub('_warped_seg.npy','_warped.pickle',_file))
                            else:
                                print('not matched')

  0%|          | 0/4 [00:00<?, ?it/s]

28
16


/tmp/ipykernel_1972170/923751762.py:27: UserWarning:

../TH1-CHR2_Small_Volumes/Female1_slices/276_cap_1_0001_0_single_skel_sing.tif is a low contrast image

 25%|██▌       | 1/4 [01:23<04:09, 83.23s/it]/tmp/ipykernel_1972170/923751762.py:27: UserWarning:

../TH1-CHR2_Small_Volumes/Female1_slices/304_arteriole_1_0001_0_single_skel_sing.tif is a low contrast image



56
22
not matched
not matched
not matched
not matched
not matched
not matched


 50%|█████     | 2/4 [01:53<01:44, 52.10s/it]/tmp/ipykernel_1972170/923751762.py:27: UserWarning:

../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_0_single_skel_sing.tif is a low contrast image



51
21


 75%|███████▌  | 3/4 [02:12<00:37, 37.04s/it]/tmp/ipykernel_1972170/923751762.py:27: UserWarning:

../TH1-CHR2_Small_Volumes/Female1_slices/57_art_1_0001_0_single_skel_sing.tif is a low contrast image



23
11
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matched
not matche

100%|██████████| 4/4 [03:43<00:00, 55.81s/it]


In [30]:
skel.shape

(512, 100, 32)

In [31]:
directory = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
files = directory.glob('*_warped.pickle')
files = sorted([x.as_posix() for x in files])
print(len(files))

2932


In [81]:
res_num_edges = []
res_num_nodes = []
res_mean_path_length = []
res_num_terminal_nodes = []
for file in tqdm(files):
    graph = nx.read_gpickle(file)
    res_num_edges.append(len(graph.edges))
    res_num_nodes.append(len(graph.nodes))
    res_mean_path_length.append(np.mean(nx.to_pandas_edgelist(graph)['weight']))
    res_num_terminal_nodes.append(np.sum([d == 1 for n,d in graph.degree()]))

100%|██████████| 2932/2932 [00:15<00:00, 192.91it/s]


In [82]:
print(np.mean(res_num_edges))
print(np.std(res_num_edges))
print(np.mean(res_num_nodes))
print(np.std(res_num_nodes))
print(np.mean(res_mean_path_length))
print(np.std(res_mean_path_length))
print(np.mean(res_num_terminal_nodes))
print(np.std(res_num_terminal_nodes))

16.281036834924965
4.484794138098887
25.15825375170532
6.08877509985768
67.58601362199684
13.943615400888259
21.246930422919508
7.273858104449935


# write vessel measurments to graph files

In [9]:
directory = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
files = directory.glob('*_warped.pickle')
files = sorted([x.as_posix() for x in files])

print(len(files))
files = [x for x in files if not os.path.exists(re.sub('.pickle','_radii.pickle',x))]
print(len(files))
#files = [x for x in files if not os.path.exists(re.sub('.pickle','_radii_forepaw.pickle',x))]
np.random.shuffle(files)
print(len(files))
file = files[0]

2932
0
0


IndexError: list index out of range

In [10]:
file
os.path.exists(re.sub('_upsampled_seg','_upsampled',re.sub('_warped.pickle','_warped_mean.npy',file)))

NameError: name 'file' is not defined

In [11]:
min_prob = 0.5
#max_var = 0.1
sampling = 1/5

for file in tqdm(files):
    if not os.path.exists(re.sub('.pickle','_radii.pickle',file)):
        graph = nx.read_gpickle(file)
        print(len(graph.edges))
        if len(graph.edges) < 1500:
                            img_file = re.sub('_warped.pickle','_warped.tif',file)
                            seg_file = re.sub('_warped.pickle','_warped_seg.npy',file)
                            mean_file = re.sub('_upsampled_seg','_upsampled',re.sub('_warped.pickle','_warped_mean.npy',file))
                            #std_file = re.sub('_upsampled_seg','_upsampled',re.sub('_warped.pickle','_warped_2x_std.npy',file))
                            img = io.imread(img_file)
                            img_ch2 = sp.ndimage.zoom(np.swapaxes(img[:,1,:,:],0,2),(1,1,2.6))
                            img = sp.ndimage.zoom(np.swapaxes(img[:,0,:,:],0,2),(1,1,2.6))
                            seg = np.load(seg_file)
                            mean = np.load(mean_file)
                            #std = np.load(std_file)
                            seg_dst = distance_transform_edt(seg)
                            #nrn_dst = np.load(re.sub('_warped.pickle','_warped_seg_nrn_dst.npy',file))
                            a, b, c = np.mgrid[-15:16:1, -15:16:1, -15:16:1]
                            abc = np.dstack([a.flat,b.flat, c.flat])
                            mu = np.array([0,0,0])
                            sigma = np.array([0.636,0.127,0.127])
                            covariance = np.diag(sigma**2)
                            d = multivariate_normal.pdf(abc, mean=mu, cov=covariance)
                            d = d.reshape((len(a),len(b),len(c)))
                            deconv_img = np.copy(img)
                            deconv_img =  1023 * restoration.richardson_lucy(img/1023.0, d, iterations=10) - 1023 * restoration.richardson_lucy(img_ch2/1023.0, d, iterations=10)
                            deconv_img = np.int16(deconv_img)
                            
                            for i in tqdm(range(len(graph.edges))):
                                path = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['pts']
                                _pred_radii = np.mean(seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
                                _pred_radii_max = np.max(seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
                                if _pred_radii == 0:
                                    _pred_radii =1
                                
                                _box_fit = max([np.int16(_pred_radii_max)+10, 15])
                                #path_grad = np.gradient(path,edge_order=2)[0]
                                path_smooth = np.float32(np.copy(path))
                                for k in range(len(path[0])):
                                    path_smooth[:,k] = sp.ndimage.gaussian_filter1d(np.float64(path[:,k]),3,mode='nearest')
                                path_grad = np.gradient(path_smooth,edge_order=2)[0]
                                res_fwhm = []
                                X = np.arange(-1*_box_fit,_box_fit+1,1)
                                Y = np.arange(-1*_box_fit,_box_fit+1,1)
                                x,y = np.meshgrid(X,Y)
                                x = x.flatten()
                                y = y.flatten()
                                z = np.zeros(len(x))
                                xy = np.vstack([x,y,z])
                                
                                
                                res_fwhm = []
                                
                                res_fwhm_sigma = []
                                
                                def calc_fwhm_path(I):
                                    point_grad = path_grad[I]
                                    point = path[I]
                                    if all(point_grad[0:2] == [0,0]) and abs(point_grad[2]/point_grad[2]) == 1:
                                        rotated = xy.T + point
                                    else:
                                        rotated = _rotmat(point_grad,xy.T) + point
                                    points_img = sp.ndimage.map_coordinates(#np.int16(mean[1]*1024),
                                                                        deconv_img,
                                                                        rotated.T, 
                                                                        order=3,
                                                                        mode='constant')
                                    
                                            
                                    points_img = np.reshape(points_img,(len(X),len(Y)))
                                    points_img_no_smooth = np.copy(points_img)
                                    points_img = sp.ndimage.gaussian_filter(points_img, sigma = _pred_radii*.4)

                                    
                                    
                                    _point = np.array(np.arange(0,_pred_radii+20,sampling))
                                    _zeros = np.zeros(len(_point))
                                    _point = np.array([_point,_zeros])
                                    _centre = closest_node([len(X)//2+1,len(Y)//2+1],peak_local_max(points_img.T))
                                    
                                    _res = []
                                    
                                    #fig,ax = plt.subplots(1)
                                    #ax.set_aspect('equal')
                                    #ax.imshow(points_img_no_smooth,
                                    #          vmin = 0,
                                    #          vmax = np.max(deconv_img))
                                    #ax.scatter(_centre[0],_centre[1])
                                    for deg in np.arange(0,360,10):
                                        rot_point = np.dot(np.array([[np.cos(np.deg2rad(deg)),-1*np.sin(np.deg2rad(deg))],[np.sin(np.deg2rad(deg)),np.cos(np.deg2rad(deg))]]),_point)
                                        rot_point[0] = rot_point[0] + _centre[0]
                                        rot_point[1] = rot_point[1] + _centre[1]
                                        points_vals = sp.ndimage.map_coordinates(points_img.T,
                                                                                 rot_point, 
                                                                                 order=3,
                                                                                 cval=0)
                                        points_vals = sp.ndimage.gaussian_filter1d(points_vals,sigma=_pred_radii*.4/sampling)
                                        points_vals_grad = np.gradient(points_vals)
                                        _ = np.array(sp.signal.argrelextrema(points_vals + np.random.uniform(-1e-5,1e-5,len(points_vals)),np.less,order=3))
                                        if _.shape[1] != 0:
                                            points_vals_grad = np.gradient(points_vals[:_[0,0]+3])
                                            _ = np.argmin(points_vals_grad)
                                            _res.append(_*sampling)
                                            #ax.scatter(_*sampling*np.cos(np.deg2rad(deg))+_centre[0],_*sampling*np.sin(np.deg2rad(deg))+_centre[1],color='r')
                                        else:
                                            points_vals_grad = np.gradient(points_vals)
                                            _ = np.argmin(points_vals_grad)
                                            _res.append(_*sampling)
                                            #ax.scatter(_*sampling*np.cos(np.deg2rad(deg))+_centre[0],_*sampling*np.sin(np.deg2rad(deg))+_centre[1],color='r')
                                    _res = np.array(_res)
                                    _res = _res[~np.isnan(_res)]
                                    _res = _res[np.where(_res!=0)]
                                    _mean = np.mean(_res)
                                    _std = np.std(_res)
                                    _mask = np.where(np.logical_and(_res>_mean-2*_std, _res<_mean+2*_std))
                                    _res = _res[_mask]
                                    radii = np.mean(_res)
                                    radii_std = np.std(_res)
                                    circ = Circle(_centre,radii,fill=False)
                                    #ax.add_patch(circ)
                                    #ax.set_title(radii)
                                    #fig.savefig('/home/rozakmat/scratch/_tmp/'+re.sub('matt_raw_warped_upsampled_seg/','',re.sub('_warped.pickle','',file))+str(i)+'_'+str(I)+'.png')
                                    #plt.clf()
                                    
                                        
                                    return radii, radii_std
                                
                                pool = multiprocessing.Pool(16)
                                _vals, _vals_sigma = zip(*pool.map(calc_fwhm_path, range(len(path))))
                                #images = []
                                #for I in tqdm(range(len(path))):
                                #    images.append(imageio.imread('/home/rozakmat/scratch/_tmp/'+re.sub('matt_raw_warped_upsampled_seg/','',re.sub('_warped.pickle','',file))+str(i)+'_'+str(I)+'.png'))
                                #    os.remove('/home/rozakmat/scratch/_tmp/'+re.sub('matt_raw_warped_upsampled_seg/','',re.sub('_warped.pickle','',file))+str(i)+'_'+str(I)+'.png')
                                #imageio.mimsave('/home/rozakmat/scratch/gifs/'+re.sub('matt_raw_warped_upsampled_seg/','',re.sub('_warped.pickle','',file))+str(i)+'.gif', images, format='GIF', fps=2)
                                
                                #_nrn_dst_vals = nrn_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii'] = np.mean(_vals)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_std'] = np.std(_vals)
                                #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['mean_neuron_distance'] = np.mean(_nrn_dst_vals)
                                #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['neuron_distance_std'] = np.std(_nrn_dst_vals)
                                #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['neuron_distance_min'] = np.min(_nrn_dst_vals)
                                #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['gender'] = gender
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights'] = _vals
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights_uncertanty'] = _vals_sigma
                                #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight'] = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight']
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0z'] = path[0][0]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0y'] = path[0][1]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0x'] = path[0][2]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1z'] = path[-1][0]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1y'] = path[-1][1]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1x'] = path[-1][2]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['mean_depth'] = np.mean(path[:,0])
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['euclidean-dst'] = np.sqrt(np.sum(np.square(path[-1]-path[0])))
                                #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['subject'] = subj
                                #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['treatment'] = treatment
                                #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['wavelength'] = wavelength
                                #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['power'] = power_per
                                #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['age'] = age
                                #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['days_post_injury'] = days_post_injury
                            nx.write_gpickle(graph, re.sub('.pickle','_radii.pickle',file))

0it [00:00, ?it/s]


In [12]:
files = sorted(files)
#files

In [28]:
re.sub('matt_raw_warped_single_upsampled_seg','matt_raw_warped',re.sub('_warped.pickle','_warped.tif',file))

'matt_raw_warped/20220129_67-XYZres395_0001_warped.tif'

# Get pickle files with vascular measurments

In [51]:
directory = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
files = directory.glob('*_warped_radii.pickle')
files = sorted([x.as_posix() for x in files])
#files2 = directory.glob('*_radii_forepaw.pickle')
#files2 = sorted([x.as_posix() for x in files2])
#files += files2
files = sorted(files)
print(len(files))
#print(files2)
#files = [x for x in files if '45-' in x]
#print(sorted(files))

2932


In [52]:
file = files[0]
print(file)
'_'.join(re.sub('../TH1-CHR2_Small_Volumes/Female1_slices/','', file).split('_')[0:3])[:-2]

../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_0_warped_radii.pickle


'128_cap'

In [50]:
min_prob = 0.75
max_var = 0.1
sampling = 1/5
#xls = pd.ExcelFile('TBI_STIM_metalog_local.xlsx')
#xls2 = pd.ExcelFile('../TBI_monai_UNET/p3_metalog.xlsx')
#df = {}
#for sheet_name in xls.sheet_names:
#    df[sheet_name] = xls.parse(sheet_name)
#for sheet_name in xls2.sheet_names:
#    df[sheet_name] = xls2.parse(sheet_name)
for file in tqdm(files[::-1]):
    graph = nx.read_gpickle(file)
    #for sheet_name in xls.sheet_names + xls2.sheet_names:
        #if re.sub('_0001','',re.sub('matt_preds_graphs_fwhm_single/','',re.sub('_warped_radii.pickle','',re.sub('_warped_radii_forepaw.pickle','',re.sub('xyz','XYZ',file)))).split('-')[1]) in df[sheet_name].values:
            #subj = sheet_name
            #if (re.sub('matt_preds_graphs_fwhm_single/','',re.sub('_warped.pickle','',file)).split('-')[0].split(' ') + [''])[0] in sheet_name or (re.sub('matt_preds_graphs_fwhm_single/','',re.sub('_warped.pickle','',file)).split('-')[0].split('_') + [''])[1] in sheet_name:
    scan = '_'.join(re.sub('../TH1-CHR2_Small_Volumes/Female1_slices/','', file).split('_')[0:3])
    count = 0
    for edge in graph.edges:
        graph[edge[0]][edge[1]]['scan'] = scan
        graph[edge[0]][edge[1]]['location'] = scan[:-2]
        graph[edge[0]][edge[1]]['ID'] = '_'.join(scan.split('_')[0:2]) + str(count)
        count +=1
    #print(np.max(_res))
    #print(gender)
    #print(subj)
    #print(file)
    nx.write_gpickle(graph, re.sub('.pickle','_amended.pickle',file))
re.sub('.pickle','_amended.pickle',file)

100%|██████████| 2932/2932 [02:56<00:00, 16.63it/s]


'../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_0_warped_radii_amended.pickle'

In [53]:
#'_'.join(scan.split('_')[0:2]) + str(count)

In [54]:
#list(dic.keys())

# Grab labels from James labelled AVC for his data

In [55]:
directory = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
files = directory.glob('*_amended.pickle')
#files = directory.glob('*_with_branch_order.pickle')
files = sorted([x.as_posix() for x in files])
print(len(files))
#files

2932


# convert graphs to excel files

In [59]:
directory = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
_files = directory.glob('*_warped_radii.pickle')
_files = sorted([x.as_posix() for x in _files])
_files = sorted(_files)
print(len(_files))

directory = Path('../TH1-CHR2_Small_Volumes/Female1_slices')
files = directory.glob('*_radii*amended.pickle')
files = sorted([x.as_posix() for x in files])
#files = [x for x in files if any(re.sub('matt_preds_graphs_fwhm_single_excel/','',re.sub('_amended','',files[0])) not in y for y in _files)]
print(len(files))
files[0:10]

2932
2932


['../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_0_warped_radii_amended.pickle',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_100_warped_radii_amended.pickle',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_101_warped_radii_amended.pickle',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_102_warped_radii_amended.pickle',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_103_warped_radii_amended.pickle',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_104_warped_radii_amended.pickle',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_105_warped_radii_amended.pickle',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_106_warped_radii_amended.pickle',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_107_warped_radii_amended.pickle',
 '../TH1-CHR2_Small_Volumes/Female1_slices/128_cap_1_0001_108_warped_radii_amended.pickle']

In [64]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

res=[]
#np.random.shuffle(files)
graph = nx.read_gpickle(files[0])
df = nx.to_pandas_edgelist(graph)
if '_0001' in files[0]:
    df['scan'] = df['scan'] + '_0001'
    df['stim'] = 'post'
df['time'] = re.sub('_0001','',files[0]).split('/')[-1].split('_')[3]
for file in tqdm(files[1:]):
    graph = nx.read_gpickle(file)
    edge_df = nx.to_pandas_edgelist(graph)
    
    if '_0001' in file:
        edge_df['scan'] = edge_df['scan'] + '_0001'
        edge_df['stim'] = 'post'
    else:
        edge_df['stim'] = 'pre'
    edge_df['time'] = re.sub('_0001','',file).split('/')[-1].split('_')[3]
    df = df.append(edge_df, ignore_index=True)

    #edge_df.to_excel(re.sub('matt_preds_graphs_fwhm_single','matt_preds_graphs_fwhm_single_excel',re.sub('.pickle','.xlsx',file)))
    #res.append(len(edge_df))
    #print(edge_df.shape

100%|██████████| 2931/2931 [00:39<00:00, 73.42it/s]


In [65]:
df

,source,target,location,end-1x,radii_std,ID,end-0x,end-0z,radii,path_weights,...,path_weights_uncertanty,end-1y,mean_depth,weight,euclidean-dst,end-1z,end-0y,scan,stim,time
0,0,1,128_cap,41,0.242195,128_cap0,13,7,1.655158,"(1.572222222222222, 1.4444444444444449, 1.4294...",...,"(0.41674073415754886, 0.18921540406584889, 0.3...",77,13.513514,51.315650,42.355637,14,46,128_cap_1_0001,post,0
1,2,4,128_cap,17,0.436825,128_cap1,4,38,1.930099,"(1.6971428571428568, 1.7125000000000001, 1.972...",...,"(0.25910363540010584, 0.42701727131346806, 0.4...",110,59.175676,103.024335,65.741920,81,62,128_cap_1_0001,post,0
2,3,6,128_cap,46,0.439350,128_cap2,4,63,1.134007,"(0.8181818181818182, 1.2411764705882353, 0.760...",...,"(0.20516740460376676, 0.6821754333009274, 0.23...",44,73.437500,94.479695,60.382117,102,25,128_cap_1_0001,post,0
3,5,14,128_cap,17,0.440368,128_cap3,16,100,1.768766,"(2.070588235294118, 1.964705882352941, 2.11764...",...,"(0.673677096575078, 0.47395851949373385, 0.694...",79,140.938144,124.377732,86.706401,186,68,128_cap_1_0001,post,0
4,7,8,128_cap,13,0.386318,128_cap4,11,110,1.345884,"(1.1235294117647054, 1.464705882352941, 1.8352...",...,"(0.5291829570915192, 0.6014112583998606, 0.812...",5,128.333333,57.336830,36.455452,145,15,128_cap_1_0001,post,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47731,10,11,57_art,22,0.331275,57_art6,12,198,2.516753,"(3.165714285714286, 2.7588235294117647, 2.4666...",...,"(1.0949084390011425, 1.0230052748581226, 0.747...",94,205.338710,75.651268,53.572381,211,43,57_art_3,pre,9
47732,13,14,57_art,23,1.044262,57_art7,10,277,8.403477,"(9.627777777777776, 9.896969696969698, 10.0, 9...",...,"(1.9596784190710579, 0.6032662854850572, 0.877...",45,301.557692,59.312980,51.903757,327,50,57_art_3,pre,9
47733,14,18,57_art,27,1.343611,57_art8,23,328,4.768834,"(4.548387096774193, 7.0, 6.958823529411765, 7....",...,"(3.3019906965092294, 3.525491584033482, 4.7467...",2,387.108696,162.545905,133.824512,455,44,57_art_3,pre,9
47734,14,16,57_art,23,0.561632,57_art9,23,328,8.202435,"(9.481481481481481, 9.344827586206897, 9.72121...",...,"(5.519865607397624, 4.835186136052415, 4.83846...",91,346.530612,63.672447,57.801384,363,45,57_art_3,pre,9


In [66]:
np.unique(df['time'])

array(['0', '1', '10', '100', '101', '102', '103', '104', '105', '106',
       '107', '108', '109', '11', '110', '111', '112', '113', '114',
       '115', '116', '117', '118', '119', '12', '120', '121', '122',
       '123', '124', '125', '126', '127', '128', '129', '13', '130',
       '131', '132', '133', '134', '135', '136', '137', '138', '139',
       '14', '140', '141', '142', '143', '144', '145', '146', '147',
       '148', '149', '15', '150', '151', '152', '153', '154', '155',
       '156', '157', '158', '159', '16', '160', '161', '162', '163',
       '164', '165', '166', '167', '168', '169', '17', '170', '171',
       '172', '173', '174', '175', '176', '177', '178', '179', '18',
       '180', '181', '182', '183', '184', '185', '186', '187', '188',
       '189', '19', '190', '191', '2', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35',
       '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45',
       '46', '47'

In [71]:
#df['mean_vessel_radii_baseline'] = np.nan
#df['std_vessel_radii_baseline'] = np.nan
#df['num_vessel_radii_baseline'] = np.nan
#df['vertex_id_base'] = np.nan
#for i in tqdm(np.unique(df['ID'])):
#    df['mean_vessel_radii_baseline'].iloc[np.where(df['ID']==i)] = np.mean(df.iloc[np.where(df['ID']==i)][~df.iloc[np.where(df['ID']==i)]['stim'].str.contains('pre')]['radii'])
#    df['std_vessel_radii_baseline'].iloc[np.where(df['ID']==i)] = np.std(df.iloc[np.where(df['ID']==i)][~df.iloc[np.where(df['ID']==i)]['stim'].str.contains('pre')]['radii'])
#    df['num_vessel_radii_baseline'].iloc[np.where(df['ID']==i)] = len(df.iloc[np.where(df['ID']==i)][~df.iloc[np.where(df['ID']==i)]['stim'].str.contains('pre')]['radii'])
#    for j in list(np.where(df['ID']==i))[0]:
#        #print(j)
#        df['vertex_id_base'].iloc[j] = list(np.arange(0,len(list(df['path_weights'].iloc[np.where(df['ID']==i)])[0])))

In [24]:
n#p.repeat(tuple(),len(df['vertex_id_base'].iloc[np.where(df['vessel_id']==i-1)]),axis=0)    

array([], dtype=float64)

In [26]:
#df.iloc[0]

In [21]:
# Create a mask for rows containing '_0001'
mask = df['scan'].str.contains('_0001')
exit 
# Set 'delta_radii' and 'timepoint' for rows without '_0001'
df.loc[~mask, ['delta_radii', 'timepoint']] = [0, 0]

# Set 'timepoint' for rows with '_0001'
df.loc[mask, 'timepoint'] = 1

# Find unique scan values
unique_scans = np.unique(df['scan'])

# Iterate over the rows with '_0001'
for i in tqdm(df[mask].index):
    scan_without_suffix = df.at[i, 'scan'].replace('_0001', '')
    if scan_without_suffix in unique_scans:
        # Find the corresponding row index for the matching scan and vessel_id
        match_mask = (df['scan'] == scan_without_suffix) & (df['vessel_id'] == df.at[i, 'vessel_id'])
        match_idx = df[match_mask].index
        if not match_idx.empty:
            # Calculate 'delta_radii' using the matching row index
            df.at[i, 'delta_radii'] = df.at[i, 'radii'] - df.at[match_idx[0], 'radii']
        else:
            df.at[i, 'delta_radii'] = np.nan
    else:
        df.at[i, 'delta_radii'] = np.nan

100%|██████████| 163812/163812 [1:00:09<00:00, 45.38it/s]


In [ ]:
#df['delta_radii'] = 0
#df['timepoint'] = '_'
#for i in tqdm(range(len(df))):
#    if '_0001' not in df.iloc[i]['scan']:
#        df['delta_radii'].iloc[i] = 0
#        df['timepoint'].iloc[i] = 'pre-stimulus'
#    elif '_0001' in df.iloc[i]['scan']:
#        df['timepoint'].iloc[i] = 'post-stimulus'
#        if re.sub('_0001','',df.iloc[i]['scan']) in np.unique(df['scan']):
#            df['delta_radii'].iloc[i] = df['radii'].iloc[i] - df['radii'].iloc[np.where((df['scan'] == re.sub('_0001','',df.iloc[i]['scan'])) & (df['vessel_id']==df.iloc[i]['vessel_id']))]
#        elif re.sub('_0001','',df.iloc[i]['scan']) not in np.unique(df['scan']):
#            df['delta_radii'].iloc[i] = "NA"
#        else:
#            print('error')
#            break
#    else:
#        print('error')
#        break

In [28]:
np.sum(df["timepoint"])

163812.0

In [72]:
df.to_excel('results_sv.xlsx')

In [30]:
#df = pd.read_excel('matt_preds_graphs_fwhm_single_excel/results.xlsx')

In [31]:
df_sham = df[df['treatment']=='SHAM']
np.unique(df_sham['treatment'])

array(['SHAM'], dtype=object)

In [23]:
scans = sorted(list(np.unique(df['scan'])))
_scans = [re.sub('vbm','',re.sub('TBI','',re.sub('SHAM','',re.sub('_3D','',x)))) + '_warped' for x in scans]
for file in tqdm(files[::-1]):
    graph = nx.read_gpickle(file)
    _file = re.sub(' 2020_',' ',re.sub('-xyz','_xyz',re.sub('-XYZ','_XYZ',file)))
    if ' ' in _file:
        _file = _file.split(' ')[0] + '_' + _file.split(' ')[-1]
    match_file = list(filter(lambda str: str in _file, _scans))
    #print(file)
    #print(match_file)
    if len(match_file) == 1:
        scan = np.array(scans)[np.array(_scans) == match_file[0]][0]
        df_subset = df[df["scan"] == scan]
        if len(df_subset) == len(graph.edges):
            #print(df_subset)
            for i in range(len(df_subset)):
                graph[df_subset.iloc[i]['source']][df_subset.iloc[i]['target']]['mean_vessel_radii_baseline'] = df_subset.iloc[i]['mean_vessel_radii_baseline']
                graph[df_subset.iloc[i]['source']][df_subset.iloc[i]['target']]['std_vessel_radii_baseline'] = df_subset.iloc[i]['std_vessel_radii_baseline']
                graph[df_subset.iloc[i]['source']][df_subset.iloc[i]['target']]['num_vessel_radii_baseline'] = df_subset.iloc[i]['num_vessel_radii_baseline']
                graph[df_subset.iloc[i]['source']][df_subset.iloc[i]['target']]['delta_radii'] = df_subset.iloc[i]['delta_radii']
                graph[df_subset.iloc[i]['source']][df_subset.iloc[i]['target']]['timepoint'] = df_subset.iloc[i]['timepoint']
            nx.write_gpickle(graph,re.sub('.pickle','_full.pickle',file))

100%|██████████| 784/784 [13:52<00:00,  1.06s/it] 


In [33]:
#df.iloc[0]['pts']

In [83]:
df = df.drop('path_weights_nrn',axis=1)
df = df.drop('path_weights_uncertanty',axis=1)
df = df.drop('path_weights',axis=1)
df = df.drop('pts',axis=1)
len(df.keys())

38

In [84]:
df.to_csv('matt_preds_graphs_fwhm_single_excel/results_smaller.csv')

In [85]:
#df.iloc[0]

In [86]:
np.unique(df['start_depth'])

array([  0.  , 247.95, 249.2 , 249.85, 250.8 , 500.  ])

In [24]:
df_sham = df[df['treatment']!='VBM']
df_sham = df_sham.drop('neuron_distance_min',axis=1)
df_sham = df_sham.drop('neuron_distance_std',axis=1)
df_sham = df_sham.drop('end-0z',axis=1)
df_sham = df_sham.drop('end-0y',axis=1)
df_sham = df_sham.drop('end-0x',axis=1)
df_sham = df_sham.drop('end-1z',axis=1)
df_sham = df_sham.drop('end-1y',axis=1)
df_sham = df_sham.drop('end-1x',axis=1)
#df_sham = df_sham.drop('pts',axis=1)
df_sham = df_sham.drop('mean_depth',axis=1)
df_sham = df_sham.drop('delta_radii',axis=1)
df_sham = df_sham.drop('num_vessel_radii_baseline',axis=1)
df_sham = df_sham.drop('path_weights_uncertanty',axis=1)
#df_sham = df_sham.drop('start_depth',axis=1)
df_sham = df_sham.drop('source',axis=1)
df_sham = df_sham.drop('target',axis=1)
df_sham = df_sham.drop('radii_std',axis=1)
df_sham = df_sham.drop('days_post_injury',axis=1)
df_sham = df_sham.drop('mean_vessel_radii_baseline',axis=1)
df_sham = df_sham.drop('std_vessel_radii_baseline',axis=1)
df_sham = df_sham.drop('current',axis=1)
df_sham = df_sham.drop('frequency',axis=1)
df_sham = df_sham.drop('mean_neuron_distance',axis=1)
df_sham = df_sham.drop('path_weights_nrn_dst',axis=1)
df_sham.iloc[0]

subject                                                      C5703_3D
vessel_id                                                       12334
start_weight                                                     42.0
wavelength                                                        552
power                                                               9
path_weights_nrn    [131.0, 130.4, 129.9, 130.2, 130.6, 130.6, 132...
path_weights        (0.9757575757575756, 0.723529411764706, 0.8625...
gender                                                             NA
scan                                               C5703_3D_XYZres462
treatment                                                         C57
euclidean-dst                                               82.957821
weight                                                     138.912901
start_depth                                                     500.0
imaging_weight                                                   31.0
pts                 

In [ ]:
_scans = [re.sub('vbm','',re.sub('TBI','',re.sub('SHAM','',re.sub('_3D','',x)))) for x in scans]
_scans

In [25]:
df_sham_exp = df_sham.explode(['path_weights', 'path_weights_nrn','pts'])

In [26]:
df_sham_exp.iloc[0]

subject                                                      C5703_3D
vessel_id                                                       12334
start_weight                                                     42.0
wavelength                                                        552
power                                                               9
path_weights_nrn                                                131.0
path_weights                                                 0.975758
gender                                                             NA
scan                                               C5703_3D_XYZres462
treatment                                                         C57
euclidean-dst                                               82.957821
weight                                                     138.912901
start_depth                                                     500.0
imaging_weight                                                   31.0
pts                 

In [27]:
np.stack(df_sham_exp['pts'].values)[:,0]

array([  1,   1,   1, ..., 504, 504, 504], dtype=int16)

In [28]:
df_sham_exp['a'] = np.stack(df_sham_exp['pts'].values)[:,0]
df_sham_exp['b'] = np.stack(df_sham_exp['pts'].values)[:,1]
df_sham_exp['c'] = np.stack(df_sham_exp['pts'].values)[:,2]
df_sham_exp = df_sham_exp.drop('pts',axis=1)

In [29]:
df_sham_exp['vertex_id'] = df_sham_exp['vessel_id'].astype('str') + '_' + df_sham_exp['a'].astype('str') + '_' + df_sham_exp['b'].astype('str') + '_' + df_sham_exp['c'].astype('str')

In [30]:
df_sham_exp.to_csv('matt_preds_graphs_fwhm_single_excel/results_vertexwise.csv')

In [46]:
df_sham_exp['vertex_id']

0           16419_1_278_153
0           16419_1_278_152
0           16419_1_278_151
0           16419_1_278_150
0           16419_1_277_149
                ...        
326435    30130_510_203_137
326435    30130_509_202_138
326435    30130_509_202_139
326435    30130_509_202_140
326435    30130_509_202_141
Name: vertex_id, Length: 6481061, dtype: object

In [45]:
df_sham_exp.keys()

Index(['weight', 'euclidean-dst', 'path_weights_nrn', 'power', 'gender',
       'imaging_weight', 'vessel_id', 'path_weights', 'start_weight', 'radii',
       'treatment', 'subject', 'scan', 'wavelength', 'age', 'start_depth',
       'stim', 'timepoint', 'a', 'b', 'c'],
      dtype='object')

# radii via path gradient

In [ ]:
from matplotlib.patches import Circle
from skimage.feature import peak_local_max
import imageio
plt.rcParams['figure.figsize'] = [15, 15]

def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.sum((nodes - node)**2, axis=1)
    if np.min(dist_2)>10:
        return node
    else:
        return nodes[np.argmin(dist_2)]

sampling = 1/5

min_prob = 0.75
max_var = 0.1
for i in tqdm(range(len(graph.edges))):
    path = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['pts']
    _pred_radii = np.mean(seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
    _pred_radii_max = np.max(seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
    if _pred_radii == 0:
        _pred_radii =1
    _pred_radii_0001 = np.mean(seg_0001_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
    _pred_radii_max_0001 = np.max(seg_0001_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
    if _pred_radii_0001 == 0:
        _pred_radii_0001 =1
    
    _box_fit = max([np.int16(_pred_radii_max)+15, np.int16(_pred_radii_max_0001)+15, 10])
    #path_grad = np.gradient(path,edge_order=2)[0]
    path_smooth = np.float32(np.copy(path))
    for k in range(len(path[0])):
        path_smooth[:,k] = sp.ndimage.gaussian_filter1d(np.float64(path[:,k]),3,mode='nearest')
    path_grad = np.gradient(path_smooth,edge_order=2)[0]
    res_fwhm = []
    res_fwhm_0001 = []
    X = np.arange(-1*_box_fit,_box_fit+1,1)
    Y = np.arange(-1*_box_fit,_box_fit+1,1)
    x,y = np.meshgrid(X,Y)
    x = x.flatten()
    y = y.flatten()
    z = np.zeros(len(x))
    xy = np.vstack([x,y,z])
    
    
    res_fwhm = []
    res_fwhm_0001 = []
    
    res_fwhm_sigma = []
    res_fwhm_sigma_0001 = []
    
    for I in tqdm(range(len(path))):
        point_grad = path_grad[I]
        point = path[I]
        if all(point_grad[0:2] == [0,0]) and abs(point_grad[2]/point_grad[2]) == 1:
            rotated = xy.T + point
        else:
            rotated = _rotmat(point_grad,xy.T) + point
        points_img = sp.ndimage.map_coordinates(#np.int16(mean[1]*1024),
                                            deconv_img,
                                            rotated.T, 
                                            order=1,
                                            mode='constant')
        points_img_0001 = sp.ndimage.map_coordinates(#np.int16(mean_0001[1]*1024),
                                                 deconv_img_0001,
                                                 rotated.T, 
                                                 order=1,
                                                 mode='constant')
                
        points_img = np.reshape(points_img,(len(X),len(Y)))
        points_img = sp.ndimage.gaussian_filter(points_img, sigma = np.min([_pred_radii,_pred_radii_0001])*.4)
        #points_img = sp.ndimage.median_filter(points_img, size = np.int32(np.max([np.min([_pred_radii,_pred_radii_0001])*.4,1])))
        points_img_0001 = np.reshape(points_img_0001,(len(X),len(Y)))
        points_img_0001 = sp.ndimage.gaussian_filter(points_img_0001, sigma = np.min([_pred_radii,_pred_radii_0001])*.4)
        #points_img_0001 = sp.ndimage.median_filter(points_img_0001, size = np.int32(np.max([np.min([_pred_radii,_pred_radii_0001])*.4,1])))
        
        _point = np.array(np.arange(0,np.max([_pred_radii,_pred_radii_0001])+10,sampling))
        _zeros = np.zeros(len(_point))
        _point = np.array([_point,_zeros])
        _centre = closest_node([len(X)//2+1,len(Y)//2+1],peak_local_max(points_img.T))
        _centre_0001 = closest_node([len(X)//2+1,len(Y)//2+1],peak_local_max(points_img_0001.T))
        
        _res = []
        
        fig,ax = plt.subplots(1)
        ax.set_aspect('equal')
        ax.imshow(points_img,
                  vmin = 0,
                  vmax = np.max(deconv_img))
        ax.scatter(_centre[0],_centre[1])
        for deg in np.arange(0,360,10):
            rot_point = np.dot(np.array([[np.cos(np.deg2rad(deg)),-1*np.sin(np.deg2rad(deg))],[np.sin(np.deg2rad(deg)),np.cos(np.deg2rad(deg))]]),_point)
            rot_point[0] = rot_point[0] + _centre[0]
            rot_point[1] = rot_point[1] + _centre[1]
            points_vals = sp.ndimage.map_coordinates(points_img.T,
                                                     rot_point, 
                                                     order=3,
                                                     cval=0)
            points_vals = sp.ndimage.gaussian_filter1d(points_vals,sigma=np.min([_pred_radii,_pred_radii_0001])*.4/sampling)
            points_vals_grad = np.gradient(points_vals)
            #plt.scatter(*rot_point,
            #            c=points_vals,
            #            vmin=np.min(points_vals), 
            #            vmax=np.max(points_vals))
            _ = np.array(sp.signal.argrelextrema(points_vals + np.random.uniform(-1e-5,1e-5,len(points_vals)),np.less,order=3))
            if _.shape[1] != 0:
                points_vals_grad = np.gradient(points_vals[:_[0,0]+3])
                _ = np.argmin(points_vals_grad)
                _res.append(_*sampling)
                ax.scatter(_*sampling*np.cos(np.deg2rad(deg))+_centre[0],_*sampling*np.sin(np.deg2rad(deg))+_centre[1],color='r')
            else:
                points_vals_grad = np.gradient(points_vals)
                _ = np.argmin(points_vals_grad)
                _res.append(_*sampling)
                ax.scatter(_*sampling*np.cos(np.deg2rad(deg))+_centre[0],_*sampling*np.sin(np.deg2rad(deg))+_centre[1],color='r')
        _res = np.array(_res)
        _res = _res[np.where(_res!=0)]
        _mean = np.mean(_res)
        _std = np.std(_res)
        _mask = np.where(np.logical_and(_res>_mean-2*_std, _res<_mean+2*_std))
        _res = _res[_mask]
        radii = np.mean(_res)
        circ = Circle(_centre,radii,fill=False)
        #ax.scatter(_centre[0],_centre[1],color='blue')
        ax.add_patch(circ)
        fig.savefig('/home/rozakmat/scratch/_tmp/'+str(i)+'_'+str(I)+'.png')
        plt.show()
        #plt.clf()
        _res_0001 = []
        fig,ax = plt.subplots(1)
        ax.set_aspect('equal')
        ax.imshow(points_img_0001,
                  vmin = 0,
                  vmax = np.max(deconv_img_0001))
        ax.scatter(_centre_0001[0],_centre_0001[1])
        for deg in np.arange(0,360,10):
            rot_point = np.dot(np.array([[np.cos(np.deg2rad(deg)),-1*np.sin(np.deg2rad(deg))],[np.sin(np.deg2rad(deg)),np.cos(np.deg2rad(deg))]]),_point)
            rot_point[0] = rot_point[0] + _centre_0001[0]
            rot_point[1] = rot_point[1] + _centre_0001[1]
            points_vals = sp.ndimage.map_coordinates(points_img_0001.T,
                                                     rot_point, 
                                                     order=3,
                                                     cval=0)
            points_vals = sp.ndimage.gaussian_filter1d(points_vals,sigma=np.min([_pred_radii,_pred_radii_0001])*.4/sampling)
            points_vals_grad = np.gradient(points_vals)
            #plt.scatter(*rot_point,
            #            c=points_vals,
            #            vmin=np.min(points_vals), 
            #            vmax=np.max(points_vals))
            _ = np.array(sp.signal.argrelextrema(points_vals + np.random.uniform(-1e-5,1e-5,len(points_vals)),np.less,order=3))
            if _.shape[1] != 0:
                points_vals_grad = np.gradient(points_vals[:_[0,0]+3])
                _ = np.argmin(points_vals_grad)
                _res_0001.append(_*sampling)
                ax.scatter(_*sampling*np.cos(np.deg2rad(deg))+_centre_0001[0],_*sampling*np.sin(np.deg2rad(deg))+_centre_0001[1],color='r')
            else:
                points_vals_grad = np.gradient(points_vals)
                _ = np.argmin(points_vals_grad)
                _res_0001.append(_*sampling)
                ax.scatter(_*sampling*np.cos(np.deg2rad(deg))+_centre_0001[0],_*sampling*np.sin(np.deg2rad(deg))+_centre_0001[1],color='r')
        _res_0001 = np.array(_res_0001)
        _res_0001 = _res_0001[np.where(_res_0001!=0)]
        _mean_0001 = np.mean(_res_0001)
        _std_0001 = np.std(_res_0001)
        _mask = np.where(np.logical_and(_res_0001>_mean_0001-2*_std_0001, _res_0001<_mean_0001+2*_std_0001))
        _res_0001 = _res_0001[_mask]
        radii_0001 = np.mean(_res_0001)
        circ = Circle(_centre_0001,radii_0001,fill=False)
        ax.add_patch(circ)
        res_fwhm.append(radii)
        res_fwhm_0001.append(radii_0001)
        #plt.imshow(points_img)
        #for i in range(len(_res)):
        #ax.scatter(_centre_0001[0],_centre_0001[1],color='blue')
        fig.savefig('/home/rozakmat/scratch/_tmp/'+str(i)+'_'+str(I)+'_0001.png')
        plt.show()
        #plt.clf()
    images = []
    for I in tqdm(range(len(path))):
        images.append(imageio.imread('/home/rozakmat/scratch/_tmp/'+str(i)+'_'+str(I)+'_0001.png'))
        os.remove('/home/rozakmat/scratch/_tmp/'+str(i)+'_'+str(I)+'_0001.png')
    imageio.mimsave('/home/rozakmat/scratch/gifs/'+str(i)+'_0001.gif', images, format='GIF', fps=2)
    for I in tqdm(range(len(path))):
        images.append(imageio.imread('/home/rozakmat/scratch/_tmp/'+str(i)+'_'+str(I)+'.png'))
        os.remove('/home/rozakmat/scratch/_tmp/'+str(i)+'_'+str(I)+'.png')
    imageio.mimsave('/home/rozakmat/scratch/gifs/'+str(i)+'.gif', images, format='GIF', fps=2)
    plt.plot(res_fwhm)
    plt.plot(res_fwhm_0001)
    plt.show()
    break

In [ ]:
len(_res)

In [ ]:
plt.plot(res_fwhm)
plt.plot(res_fwhm_0001)
print(np.mean(res_fwhm))
print(np.mean(res_fwhm_0001))
print(np.mean(res_fwhm_0001)-np.mean(res_fwhm))

In [ ]:
print(_pred_radii)
#plt.scatter(*_point + len(x)//2 +1)
_res = []
for deg in np.arange(0,360,10):
    rot_point = np.dot(np.array([[np.cos(np.deg2rad(deg)),-1*np.sin(np.deg2rad(deg))],[np.sin(np.deg2rad(deg)),np.cos(np.deg2rad(deg))]]),_point)
    rot_point[0] = rot_point[0] + _centre_0001[0]
    rot_point[1] = rot_point[1] + _centre_0001[1]
    points_vals = sp.ndimage.map_coordinates(points_img_0001.T,
                                             rot_point, 
                                             order=5,
                                             cval=0)
    points_vals = sp.ndimage.gaussian_filter1d(points_vals,sigma=_pred_radii/sampling)
    points_vals = points_vals + np.random.uniform(-1e-5,1e-5,len(points_vals))
    #plt.plot(points_vals)
    #plt.show()
    #points_vals_grad = np.gradient(points_vals)
    _ = np.array(sp.signal.argrelextrema(points_vals,np.less,order=5))
    #print(_)
    if _.shape[1] != 0:
        print(_)
        #print(points_vals)
        #print(_[0,0])
        plt.plot(points_vals[:(_[0,0]+3)])
        #plt.show()
        points_vals_grad = np.gradient(points_vals[:_[0,0]+3])
        _ = np.argmin(points_vals_grad)
        _res.append(_*sampling)
        #plt.plot(points_vals_grad)
    else:
        plt.plot(points_vals)
        #plt.show()
        points_vals_grad = np.gradient(points_vals)
        _ = np.argmin(points_vals_grad)
        _res.append(_*sampling)
        #plt.plot(points_vals_grad)
plt.show()
_res = np.array(_res)
plt.plot(_res)
plt.show()
#_mean = np.mean(_res)
#_std = np.std(_res)
#_mask = np.where(np.logical_and(_res>_mean-2*_std, _res<_mean+2*_std))
#_res = _res[_mask]
#plt.plot(_res)
#plt.show()

In [ ]:
fig,ax = plt.subplots(1)
ax.set_aspect('equal')
ax.imshow(points_img_0001)

In [ ]:
_[0,0]

In [ ]:
plt.hist(_res)
plt.show()

In [ ]:
_res = np.array(_res)
_mean = np.mean(_res)
_std = np.std(_res)
_mask = np.where(np.logical_and(_res>_mean-_std, _res<_mean+_std))
_res = _res[_mask]
plt.hist(_res)
plt.show()
#_mask

In [ ]:
for i in tqdm(range(len(graph.edges))):
    path = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['pts']
    _pred_radii = np.mean(seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
    _pred_radii_max = np.max(seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
    if _pred_radii == 0:
        _pred_radii =1
    _pred_radii_0001 = np.mean(seg_0001_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
    _pred_radii_max_0001 = np.max(seg_0001_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
    if _pred_radii_0001 == 0:
        _pred_radii_0001 =1
    
    _box_fit = max([np.int16(_pred_radii_max)+10, np.int16(_pred_radii_max_0001)+10, 10])
    #path_grad = np.gradient(path,edge_order=2)[0]
    path_smooth = np.float32(np.copy(path))
    for k in range(len(path[0])):
        path_smooth[:,k] = sp.ndimage.gaussian_filter1d(np.float64(path[:,k]),3,mode='nearest')
    path_grad = np.gradient(path_smooth,edge_order=2)[0]
    res_fwhm = []
    res_fwhm_0001 = []
    X = np.arange(-1*_box_fit,_box_fit+1,1)
    Y = np.arange(-1*_box_fit,_box_fit+1,1)
    x,y = np.meshgrid(X,Y)
    x = x.flatten()
    y = y.flatten()
    z = np.zeros(len(x))
    xy = np.vstack([x,y,z])
    
    
    res_fwhm = []
    res_fwhm_0001 = []
    
    res_fwhm_sigma = []
    res_fwhm_sigma_0001 = []
    
    def calc_fwhm_path(I):
        point_grad = path_grad[I]
        point = path[I]
        if all(point_grad[0:2] == [0,0]) and abs(point_grad[2]/point_grad[2]) == 1:
            rotated = xy.T + point
        else:
            rotated = _rotmat(point_grad,xy.T) + point
        points_img = sp.ndimage.map_coordinates(#np.int16(mean[1]*1024),
                                            deconv_img,
                                            rotated.T, 
                                            order=3,
                                            mode='constant')
        points_img_0001 = sp.ndimage.map_coordinates(#np.int16(mean_0001[1]*1024),
                                                 deconv_img_0001,
                                                 rotated.T, 
                                                 order=3,
                                                 mode='constant')
                
        points_img = np.reshape(points_img,(len(X),len(Y)))
        points_img_no_smooth = np.copy(points_img)
        points_img = sp.ndimage.gaussian_filter(points_img, sigma = np.min([_pred_radii,_pred_radii_0001])*.4)
        #points_img = sp.ndimage.median_filter(points_img, size = np.int32(np.max([np.min([_pred_radii,_pred_radii_0001])*.4,1])))
        points_img_0001 = np.reshape(points_img_0001,(len(X),len(Y)))
        points_img_0001_no_smooth = np.copy(points_img_0001)
        points_img_0001 = sp.ndimage.gaussian_filter(points_img_0001, sigma = np.min([_pred_radii,_pred_radii_0001])*.4)
        #points_img_0001 = sp.ndimage.median_filter(points_img_0001, size = np.int32(np.max([np.min([_pred_radii,_pred_radii_0001])*.4,1])))
        
        _point = np.array(np.arange(0,np.max([_pred_radii,_pred_radii_0001])+20,sampling))
        _zeros = np.zeros(len(_point))
        _point = np.array([_point,_zeros])
        _centre = closest_node([len(X)//2+1,len(Y)//2+1],peak_local_max(points_img.T))
        _centre_0001 = closest_node([len(X)//2+1,len(Y)//2+1],peak_local_max(points_img_0001.T))
        
        _res = []
        
        fig,ax = plt.subplots(1)
        ax.set_aspect('equal')
        ax.imshow(points_img_no_smooth,
                  vmin = 0,
                  vmax = np.max(deconv_img))
        ax.scatter(_centre[0],_centre[1])
        for deg in np.arange(0,360,10):
            rot_point = np.dot(np.array([[np.cos(np.deg2rad(deg)),-1*np.sin(np.deg2rad(deg))],[np.sin(np.deg2rad(deg)),np.cos(np.deg2rad(deg))]]),_point)
            rot_point[0] = rot_point[0] + _centre[0]
            rot_point[1] = rot_point[1] + _centre[1]
            points_vals = sp.ndimage.map_coordinates(points_img.T,
                                                     rot_point, 
                                                     order=3,
                                                     cval=0)
            points_vals = sp.ndimage.gaussian_filter1d(points_vals,sigma=np.min([_pred_radii,_pred_radii_0001])*.4/sampling)
            points_vals_grad = np.gradient(points_vals)
            #plt.scatter(*rot_point,
            #            c=points_vals,
            #            vmin=np.min(points_vals), 
            #            vmax=np.max(points_vals))
            _ = np.array(sp.signal.argrelextrema(points_vals + np.random.uniform(-1e-5,1e-5,len(points_vals)),np.less,order=3))
            if _.shape[1] != 0:
                points_vals_grad = np.gradient(points_vals[:_[0,0]+3])
                _ = np.argmin(points_vals_grad)
                _res.append(_*sampling)
                ax.scatter(_*sampling*np.cos(np.deg2rad(deg))+_centre[0],_*sampling*np.sin(np.deg2rad(deg))+_centre[1],color='r')
            else:
                points_vals_grad = np.gradient(points_vals)
                _ = np.argmin(points_vals_grad)
                _res.append(_*sampling)
                ax.scatter(_*sampling*np.cos(np.deg2rad(deg))+_centre[0],_*sampling*np.sin(np.deg2rad(deg))+_centre[1],color='r')
        _res = np.array(_res)
        _res = _res[np.where(_res!=0)]
        _mean = np.mean(_res)
        _std = np.std(_res)
        _mask = np.where(np.logical_and(_res>_mean-2*_std, _res<_mean+2*_std))
        _res = _res[_mask]
        radii = np.mean(_res)
        circ = Circle(_centre,radii,fill=False)
        ax.add_patch(circ)
        ax.set_title(radii)
        fig.savefig('/home/rozakmat/scratch/_tmp/'+str(i)+'_'+str(I)+'.png')
        plt.clf()
        _res_0001 = []
        fig,ax = plt.subplots(1)
        ax.set_aspect('equal')
        ax.imshow(points_img_0001_no_smooth,
                  vmin = 0,
                  vmax = np.max(deconv_img_0001))
        ax.scatter(_centre_0001[0],_centre_0001[1])
        for deg in np.arange(0,360,10):
            rot_point = np.dot(np.array([[np.cos(np.deg2rad(deg)),-1*np.sin(np.deg2rad(deg))],[np.sin(np.deg2rad(deg)),np.cos(np.deg2rad(deg))]]),_point)
            rot_point[0] = rot_point[0] + _centre_0001[0]
            rot_point[1] = rot_point[1] + _centre_0001[1]
            points_vals = sp.ndimage.map_coordinates(points_img_0001.T,
                                                     rot_point, 
                                                     order=3,
                                                     cval=0)
            points_vals = sp.ndimage.gaussian_filter1d(points_vals,sigma=np.min([_pred_radii,_pred_radii_0001])*.4/sampling)
            points_vals_grad = np.gradient(points_vals)
            _ = np.array(sp.signal.argrelextrema(points_vals + np.random.uniform(-1e-5,1e-5,len(points_vals)),np.less,order=3))
            if _.shape[1] != 0:
                points_vals_grad = np.gradient(points_vals[:_[0,0]+3])
                _ = np.argmin(points_vals_grad)
                if _ != 0:
                    _res_0001.append(_*sampling)
                    ax.scatter(_*sampling*np.cos(np.deg2rad(deg))+_centre_0001[0],_*sampling*np.sin(np.deg2rad(deg))+_centre_0001[1],color='r')
            else:
                points_vals_grad = np.gradient(points_vals)
                _ = np.argmin(points_vals_grad)
                if _ != 0:
                    _res_0001.append(_*sampling)
                    ax.scatter(_*sampling*np.cos(np.deg2rad(deg))+_centre_0001[0],_*sampling*np.sin(np.deg2rad(deg))+_centre_0001[1],color='r')
        _res_0001 = np.array(_res_0001)
        _res_0001 = _res_0001[np.where(_res_0001!=0)]
        _mean_0001 = np.mean(_res_0001)
        _std_0001 = np.std(_res_0001)
        _mask = np.where(np.logical_and(_res_0001>_mean_0001-2*_std_0001, _res_0001<_mean_0001+2*_std_0001))
        _res_0001 = _res_0001[_mask]
        radii_0001 = np.mean(_res_0001)
        circ = Circle(_centre_0001,radii_0001,fill=False)
        ax.add_patch(circ)
        ax.set_title(radii_0001)
        fig.savefig('/home/rozakmat/scratch/_tmp/'+str(i)+'_'+str(I)+'_0001.png')
        plt.clf()
            
        return radii, radii_0001
    
    pool = multiprocessing.Pool(8)
    res_fwhm, res_fwhm_0001,  = zip(*pool.map(calc_fwhm_path, range(len(path))))
    print(np.mean(res_fwhm_0001)-np.mean(res_fwhm))
    #plt.plot(res_fwhm)
    #plt.plot(res_fwhm_0001)
    #plt.show()
    images = []
    for I in tqdm(range(len(path))):
        images.append(imageio.imread('/home/rozakmat/scratch/_tmp/'+str(i)+'_'+str(I)+'_0001.png'))
        os.remove('/home/rozakmat/scratch/_tmp/'+str(i)+'_'+str(I)+'_0001.png')
    imageio.mimsave('/home/rozakmat/scratch/gifs/'+re.sub('matt_raw_warped_upsampled_seg/','',re.sub('_warped.pickle','',file))+str(i)+'_0001.gif', images, format='GIF', fps=2)
    images = []
    for I in tqdm(range(len(path))):
        images.append(imageio.imread('/home/rozakmat/scratch/_tmp/'+str(i)+'_'+str(I)+'.png'))
        os.remove('/home/rozakmat/scratch/_tmp/'+str(i)+'_'+str(I)+'.png')
    imageio.mimsave('/home/rozakmat/scratch/gifs/'+re.sub('matt_raw_warped_upsampled_seg/','',re.sub('_warped.pickle','',file))+str(i)+'.gif', images, format='GIF', fps=2)

In [ ]:
del pool

In [ ]:
_res

In [ ]:
plt.imshow(np.reshape(points,(len(X),len(Y))))
plt.colorbar()

In [ ]:
plt.imshow(np.reshape(points_std,(len(X),len(Y))))
plt.colorbar()

In [ ]:
plt.imshow(np.reshape(points_img,(len(X),len(Y))))
plt.colorbar()

In [ ]:
plt.imshow(np.reshape(points_0001,(len(X),len(Y))))
plt.colorbar()

In [ ]:
plt.imshow(np.reshape(points_std_0001,(len(X),len(Y))))
plt.colorbar()

In [ ]:
plt.imshow(np.reshape(points_img_0001,(len(X),len(Y))))
plt.colorbar()

In [ ]:
points = np.reshape(points,(len(X),len(Y)))
points_std = np.reshape(points_std,(len(X),len(Y)))
_bin = np.zeros(points.shape)
_bin[(points > min_prob) * (points_std < max_var)] = 1

In [ ]:
plt.imshow(_bin)

In [ ]:
points_0001 = np.reshape(points_0001,(len(X),len(Y)))
points_std_0001 = np.reshape(points_std_0001,(len(X),len(Y)))
_bin_0001 = np.zeros(points_0001.shape)
_bin_0001[(points_0001 > min_prob) * (points_std_0001 < max_var)] = 1

In [ ]:
plt.imshow(_bin_0001)

In [ ]:
#img = sp.ndimage.zoom(np.swapaxes(img[:,0,:,:],0,2),(1,1,2.645833333))

In [ ]:
idx = np.argwhere(img_lab)[((np.argwhere(img_lab) - [int(img_lab.shape[0]/2),int(img_lab.shape[1]/2)])**2).sum(1).argmin()]
cent_val = img_lab[idx[0],idx[1]]

In [ ]:
img_lab, N = cc3d.connected_components(_bin,return_N=True)
unique, counts = np.unique(img_lab, return_counts=True)
img_lab =  np.reshape(img_lab,(len(X),len(Y)))
cent_val = img_lab[int(img_lab.shape[0]/2),[int(img_lab.shape[1]/2)]][0]
if cent_val == 0 and np.sum(img_lab) == 0:
    area = 0
    radii = 0
elif cent_val == 0 and np.sum(img_lab) != 0:
    idx = np.argwhere(img_lab)[((np.argwhere(img_lab) - [int(img_lab.shape[0]/2),int(img_lab.shape[1]/2)])**2).sum(1).argmin()]
    cent_val = img_lab[idx[0],idx[1]]
    area = np.sum(img_lab==cent_val)
    radii = np.sqrt(area/np.pi)
else:
    area = np.sum(img_lab==cent_val)
    radii = np.sqrt(area/np.pi)

In [ ]:
area

In [ ]:
radii

In [ ]:
img_lab_0001, N_0001 = cc3d.connected_components(_bin_0001,return_N=True)
unique_0001, counts_0001 = np.unique(img_lab_0001, return_counts=True)
img_lab_0001 =  np.reshape(img_lab_0001,(len(X),len(Y)))
cent_val_0001 = img_lab_0001[int(img_lab_0001.shape[0]/2),[int(img_lab_0001.shape[1]/2)]][0]
if cent_val_0001 == 0 and np.sum(img_lab_0001) == 0:
    area_0001 = 0
    radii_0001 = 0
elif cent_val_0001 == 0 and np.sum(img_lab_0001) == 0:
    idx_0001 = np.argwhere(img_lab_0001)[((np.argwhere(img_lab_0001) - [int(img_lab_0001.shape[0]/2),int(img_lab_0001.shape[1]/2)])**2).sum(1).argmin()]
    cent_val_0001 = img_lab_0001[idx_0001[0],idx_0001[1]]
    area_0001 = np.sum(img_lab_0001==cent_val_0001)
    radii_0001 = np.sqrt(area_0001/np.pi)
else:
    area_0001 = np.sum(img_lab_0001==cent_val_0001)
    radii_0001 = np.sqrt(area_0001/np.pi)

In [ ]:
# Laplacian Radii

In [ ]:
from matplotlib.patches import Circle

sampling = 1/3

min_prob = 0.75
max_var = 0.1
for i in tqdm(range(len(graph.edges))):
    i=20
    path = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['pts']
    _pred_radii = np.mean(seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
    _pred_radii_max = np.max(seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
    if _pred_radii == 0:
        _pred_radii =1
    _pred_radii_0001 = np.mean(seg_0001_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
    _pred_radii_max_0001 = np.max(seg_0001_dst[path[::-1,0],path[::-1,1],path[::-1,2]])
    if _pred_radii_0001 == 0:
        _pred_radii_0001 =1
    
    _box_fit = max([np.int16(_pred_radii_max)+10, np.int16(_pred_radii_max_0001)+10, 10])
    #path_grad = np.gradient(path,edge_order=2)[0]
    path_smooth = np.float32(np.copy(path))
    for k in range(len(path[0])):
        path_smooth[:,k] = sp.ndimage.gaussian_filter1d(np.float64(path[:,k]),3,mode='nearest')
    path_grad = np.gradient(path_smooth,edge_order=2)[0]
    res_fwhm = []
    res_fwhm_0001 = []
    X = np.arange(-1*_box_fit,_box_fit+1,1)
    Y = np.arange(-1*_box_fit,_box_fit+1,1)
    x,y = np.meshgrid(X,Y)
    x = x.flatten()
    y = y.flatten()
    z = np.zeros(len(x))
    xy = np.vstack([x,y,z])
    
    
    res_fwhm = []
    res_fwhm_0001 = []
    
    res_fwhm_sigma = []
    res_fwhm_sigma_0001 = []
    
    for I in range(len(path)):
        point_grad = path_grad[I]
        point = path[I]
        if all(point_grad[0:2] == [0,0]) and abs(point_grad[2]/point_grad[2]) == 1:
            rotated = xy.T + point
        else:
            rotated = _rotmat(point_grad,xy.T) + point
        points_img = sp.ndimage.map_coordinates(deconv_img,
                                            rotated.T, 
                                            order=1,
                                            cval=-10000)
        points_img_0001 = sp.ndimage.map_coordinates(deconv_img_0001,
                                                 rotated.T, 
                                                 order=1,
                                                 cval=-10000)
                
        points_img = np.reshape(points_img,(len(X),len(Y)))
        points_img = sp.ndimage.gaussian_filter(points_img, sigma = np.min([_pred_radii,_pred_radii_0001])*.4)
        points_img_lp = sp.ndimage.laplace(points_img)
        plt.imshow(points_img_lp)
        plt.show()
        points_img_0001 = np.reshape(points_img_0001,(len(X),len(Y)))
        points_img_0001 = sp.ndimage.gaussian_filter(points_img_0001, sigma = np.min([_pred_radii,_pred_radii_0001])*.4)
        points_img_0001_lp = sp.ndimage.laplace(points_img_0001)
        _point = np.array(np.arange(0,np.max([_pred_radii,_pred_radii_0001])+5,sampling))
        _zeros = np.zeros(len(_point))
        _point = np.array([_point,_zeros])
        
        _res = []
        for deg in np.arange(0,360,10):
            rot_point = np.dot(np.array([[np.cos(np.deg2rad(deg)),-1*np.sin(np.deg2rad(deg))],[np.sin(np.deg2rad(deg)),np.cos(np.deg2rad(deg))]]),_point) + len(X)//2 +1
            points_vals = sp.ndimage.map_coordinates(points_img_lp,
                                                     rot_point, 
                                                     order=3,
                                                     cval=0)
            #points_vals_grad = np.gradient(points_vals)
            _ = np.array(sp.signal.argrelextrema(points_vals, np.greater))
            if _.shape[1] != 0:
                _res.append(_[0,0]*sampling)
        _res = np.array(_res)
        _mean = np.mean(_res)
        _std = np.std(_res)
        _mask = np.where(np.logical_and(_res>_mean-2*_std, _res<_mean+2*_std))
        _res = _res[_mask]
        radii = np.mean(_res)
        
        _res_0001 = []
        for deg in np.arange(0,360,10):
            rot_point = np.dot(np.array([[np.cos(np.deg2rad(deg)),-1*np.sin(np.deg2rad(deg))],[np.sin(np.deg2rad(deg)),np.cos(np.deg2rad(deg))]]),_point) + len(X)//2 +1
            points_vals = sp.ndimage.map_coordinates(points_img_0001_lp,
                                                     rot_point, 
                                                     order=3,
                                                     cval=0)
            #points_vals_grad = np.gradient(points_vals)
            _ = np.array(sp.signal.argrelextrema(points_vals, np.greater))
            if _.shape[1] != 0:
                _res_0001.append(_[0,0]*sampling)
        _res_0001 = np.array(_res_0001)
        _mean = np.mean(_res_0001)
        _std = np.std(_res_0001)
        _mask = np.where(np.logical_and(_res_0001>_mean-2*_std, _res_0001<_mean+2*_std))
        _res_0001 = _res_0001[_mask]
        radii_0001 = np.mean(_res_0001)
        res_fwhm.append(radii)
        res_fwhm_0001.append(radii_0001)
        
        #plt.imshow(points_img)
        fig,ax = plt.subplots(1)
        ax.set_aspect('equal')
        ax.imshow(points_img)
        circ = Circle((len(X)//2+1,len(Y)//2+1),_mean,fill=False)
        ax.add_patch(circ)
        for i in range(len(_res)):
            ax.scatter(_res[i]*np.cos(np.deg2rad(i*10))+len(X)//2+1,_res[i]*np.sin(np.deg2rad(i*10))+len(Y)//2+1)
        plt.show()
        fig,ax = plt.subplots(1)
        ax.set_aspect('equal')
        ax.imshow(points_img_0001)
        circ = Circle((len(X)//2+1,len(Y)//2+1),_mean,fill=False)
        ax.add_patch(circ)
        for i in range(len(_res_0001)):
            ax.scatter(_res_0001[i]*np.cos(np.deg2rad(i*10))+len(X)//2+1,_res_0001[i]*np.sin(np.deg2rad(i*10))+len(Y)//2+1)
        plt.show()
        break
    #lt.plot(res_fwhm)
    #lt.plot(res_fwhm_0001)
    #plt.show()
    break

In [ ]:
radii_0001